# AI Workshop Project

In [ ]:
# This notebook will contain the main code for the project

Sequential model

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import LSTM
def cur_model(num_Conv2D,num_filters,num_kernel,num_lstm_units,num_frames):
  model = Sequential()
  for i in range(num_Conv2D):
    model.add(Conv2D(num_filters, num_kernel, activation ='relu'))
  model.add(Reshape((num_frames, -1)))
  model.add(LSTM(num_lstm_units))